<a href="https://colab.research.google.com/github/kaushkay/midass-iiitd/blob/master/CV_Problem/%20CV_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches

In [13]:
file = open('My Drive/Colab Notebooks/CV_Problem/train_image.pkl', 'r')

data = pickle.load(file)

file.close()

FileNotFoundError: ignored